In [ ]:
import cv2
import os
import urllib.request
import pandas as pd
import requests

### df = pd.read_excel("파일이름")

In [ ]:
df = pd.read_excel("crawl_video_health.xlsx", header= None)

In [ ]:
# videoLinkArray를 생성
videoLink = df[2].tolist()
title = df[1].tolist()
# df[2]가 영상 링크를 나타내고 있음 <- 데이터에 따라 다르기에 항상 check를 진행하는 것을 권장함

In [ ]:
# FPS,frame Array initialized
fpsArray = []
frameArray = []
videoCount = len(videoLink)

In [ ]:
 # 영상 다운로드 함수 생성
def download(Url, saveName):
    if os.path.isfile(saveName):
        return
    else:
        r = requests.get(Url)
        open(saveName, "wb").write(r.content)

In [ ]:
count = 0
for i in videoLink:
    url = i
    download(Url = url, saveName = "CB" + "{}".format(count).zfill(len(str(videoCount))) + ".mp4")
    count += 1

In [ ]:
for k in range(videoCount):
    cap = cv2.VideoCapture('CB' + '{}'.format(k).zfill(len(str(videoCount))) + '.mp4')
        
    fps = cap.get(cv2.CAP_PROP_FPS)
    fCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fpsArray.append(fps)
    frameArray.append(fCount)

### Create DataFrame
#### Title: 타이틀
#### FPS: 초당 영상 프레임
#### VideoLink: 비디오 링크
#### Category: 비디오 주제
#### StartFrame: 영상 시작 프레임
#### EndFrame: 영상 종료 프레임
#### Description: 영상 설명
#### Keyword: 키워드

In [ ]:
# excel파일로 저장하기 전 pd.dataframe로 변경
# 전처리(broken_video_check)진행
dataframe = pd.DataFrame(
{
    'Title' : title,
    'VideoLink': videoLink,
    'Category': ["health"] * videoLength,
    'FPS' : fpsArray,
    'StartFrame' : [0] * videoLength,
    'EndFrame'  : frameArray
})

In [ ]:
# 영상 다운로드 과정에서 broken된 것인지 링크자체가 broken인지 확인이 필요함
# 현재 엑셀파일 속 링크들은 후자의 경우임
# broken된 영상들을 추출하여 제거
print('변경전 : {}'.format(len(dataframe)))
dataframe = dataframe[dataframe.FPS != 0]
print('변경후 : {}'.format(len(dataframe)))

In [ ]:
# to_excel()을 이용하여 excel파일로 변경
# 현재 작업중인 디렉터리로 저장
# 인덱스 없이 저장을 진행함 -> broken video에 대한 row를 제거하였기에 인덱스가 존재함
# ex) 1 -> 2 -> 4 -> 5
# 향후 순차적인 작업을 진행할 시 문제가 발생할 가능성 존재하여서 제외함
dataframe.to_excel("tagging.xlsx", index=False)